<a href="https://colab.research.google.com/github/palerique/scoliosis_llm/blob/main/scoliosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# prompt: upload a file, save it to a variable and print the name of the received file

from google.colab import files

uploaded = files.upload()

Saving sco1-5509928741_7ee89af75e_b.jpeg to sco1-5509928741_7ee89af75e_b.jpeg


In [28]:
from google.colab import userdata

from pathlib import Path
import hashlib
import google.generativeai as genai

genai.configure(api_key=userdata.get('GEMINI_API_KEY'))

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [32]:
# prompt: for each image that was uploaded previously, upload it to the generative API and ask to it the following questions:
# - describe that image
# - from 0 to 100% how sure you are that this image is a backbone x-ray image?
# - from 0 to 100% how sure you are that this image shows that someone has scoliosis?
# - Can you calculate the Cobs angles of this spine?
import google.generativeai as genai
from IPython.display import Markdown
from IPython.display import Image

jsons = []

for filename, content in uploaded.items():
  # Upload the image to the generative API
  sample_file = genai.upload_file(path=filename,
                                display_name=filename)
  # Describe the image
  response = model.generate_content(["Prepare a JSON answering the following questions about the received image: 1 - Describe the image with a medical, but simple and understandable, description. A regular person should be capable of understanding it. 2 - What is the probability that the image is a backbone x-ray image, on a scale of 0 to 100%? 3 - What is the probability that the image shows that someone has scoliosis, on a scale of 0 to 100%? 4 - What are the Cobs angles of the spine in the image?", sample_file])
  json = response.text
  jsons.append(json)

print(jsons)

['```json\n{\n  "1": "The image is an X-ray of the spine, showing a sideways curvature of the spine. The bones appear normal in terms of bone density.",\n  "2": "100%",\n  "3": "100%",\n  "4": "It is not possible to measure the Cobb angles accurately without specialized software and by a trained professional. However, visually, the curves appear to be significant."\n}\n```']


In [ ]:
genai.delete_file(sample_file.name)
print(f'Deleted {sample_file.display_name}.')